In [1]:
# Deleting the .apperception_cache if it exists, as to avoid DB conflict errors
import os
import shutil

dirpath = os.path.join('.apperception_cache')
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

# This piece of code is unsafe, and should not be run if not needed. 
# It serves for test purposes when one recieves a "dead kernel" error.
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
import sys
sys.path.append(os.path.join(os.getcwd(),"apperception"))

### IMPORTS
import lens
import point
from new_world import empty_world

# Let's define some attribute for constructing the world first
name = "trafficScene"  # world name
units = "metrics"  # world units
video_file = "amber_videos/traffic-scene-shorter.mp4"  # example video file
lens_attrs = {"fov": 120, "cam_origin": (0, 0, 0), "skew_factor": 0}
point_attrs = {"p_id": "p1", "cam_id": "cam1", "x": 0, "y": 0, "z": 0, "time": None, "type": "pos"}
camera_attrs = {"ratio": 0.5}
fps = 30

# 1. define a world
traffic_world = empty_world(name)

# 2. construct a camera
fov, res, cam_origin, skew_factor = (
    lens_attrs["fov"],
    [1280, 720],
    lens_attrs["cam_origin"],
    lens_attrs["skew_factor"],
)
cam_lens = lens.PinholeLens(res, cam_origin, fov, skew_factor)

pt_id, cam_id, x, y, z, time, pt_type = (
    point_attrs["p_id"],
    point_attrs["cam_id"],
    point_attrs["x"],
    point_attrs["y"],
    point_attrs["z"],
    point_attrs["time"],
    point_attrs["type"],
)
location = point.Point(pt_id, cam_id, (x, y, z), time, pt_type)

ratio = camera_attrs["ratio"]

# ingest the camera into the world
traffic_world = traffic_world.add_camera(
    cam_id=cam_id,
    location=location,
    ratio=ratio,
    video_file=video_file,
    metadata_identifier=name + "_" + cam_id,
    lens=cam_lens,
)

# Call execute on the world to run the detection algorithm and save the real data to the database
recognized_world = traffic_world.recognize(cam_id)

volume = traffic_world.select_intersection_of_interest_or_use_default(cam_id=cam_id)

get backend Qt5Agg


In [3]:
cams = traffic_world.get_camera()
lens = traffic_world.get_len()
# ids = traffic_world.get_id()
print("cameras are", cams)
print("lens are", lens)
# print("ids are", ids)

cameras are [('cam1', 0.5, 0.0, 0.0, 0.0, 369.5041722813606, 207.84609690826534, 120, 0.0)]
lens are [(0.5, 0.0, 0.0, 0.0, 120, 0.0)]


In [4]:
# Example Query
filtered_world = recognized_world.filter_traj_type("car").filter_traj_volume(volume).interval(0, fps * 3)
filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

Loading weights from C:\Users\youse\Desktop\Research\Apperception\apperception\apperception\../yolov5-deepsort/deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!
YOLOv5  v6.0-159-gdb6ec66 torch 1.10.2+cu113 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients


car-1 saved successfully
person-2 saved successfully
traffic light-3 saved successfully
car-4 saved successfully
car-6 saved successfully
person-7 saved successfully
traffic light-8 saved successfully
person-9 saved successfully
person-10 saved successfully
traffic light-12 saved successfully
person-12 saved successfully
person-13 saved successfully
car-14 saved successfully
person-16 saved successfully
bus-14 saved successfully
car-19 saved successfully
traffic light-10 saved successfully
traffic light-13 saved successfully
person-20 saved successfully
car-21 saved successfully
truck-21 saved successfully
bus-21 saved successfully
person-25 saved successfully
person-26 saved successfully
traffic light-26 saved successfully
traffic light-9 saved successfully
get_traj_key SELECT sq2.itemid FROM (SELECT * FROM (SELECT * FROM (SELECT * FROM item_general_trajectory WHERE worldId='5f8caac8-eb3d-4029-95b3-0433798957d3') sq0 WHERE sq0.objecttype='car') sq1 WHERE overlap(sq1.largestBbox,'STBOX

In [6]:
### Scenic Code ###
# ego = Car
# Car

### Apperception Query ###
filtered_world = recognized_world.filter_traj_type("car").interval(0, fps * 3)
filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

get_traj_key SELECT sq1.itemid FROM (SELECT * FROM (SELECT * FROM item_general_trajectory WHERE worldId='5f8caac8-eb3d-4029-95b3-0433798957d3') sq0 WHERE sq0.objecttype='car') sq1
filtered_ids are [('car-1-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-4-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-6-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-14-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-19-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-21-5f8caac8-eb3d-4029-95b3-0433798957d3',)]


In [7]:
### Scenic Code ###
# ego = Car
# Car

### Apperception Query ###
filtered_world = recognized_world.filter_traj_type("car").interval(0, fps * 3)
filtered_ids = filtered_world.get_traj_key()
print("filtered_ids are", filtered_ids)

get_traj_key SELECT sq1.itemid FROM (SELECT * FROM (SELECT * FROM item_general_trajectory WHERE worldId='5f8caac8-eb3d-4029-95b3-0433798957d3') sq0 WHERE sq0.objecttype='car') sq1
filtered_ids are [('car-1-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-4-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-6-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-14-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-19-5f8caac8-eb3d-4029-95b3-0433798957d3',), ('car-21-5f8caac8-eb3d-4029-95b3-0433798957d3',)]
